# Evaluation script with the report generation

This notebook is used to prototype the model component that can generate the 
reports on the test evaluation and provide their full documentation
in a pdf or html format.

The input specification is performed by defining the labels of individual tests, e.g. `'B1_TV1'`.
The model `ReportGenerator` component runs a loop over all tests and depending 
on the configuration, performs the evaluation for each test and adds the figures to 
the output file.

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGridTri, DICStateFields, DICCrackList, \
    DICAlignedGrid, DICGrid
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
import datetime
import collections
collections.Iterable = collections.abc.Iterable

## Crack list detector

In [ ]:
def get_dcl(test_name):
    dic_grid = DICGridTri(U_factor=100, dir_name=test_name, t=1)
    dic_grid.read_beam_design()
    dic_grid.trait_set(d_x=5, d_y=5, n_T=10);
    dsf = DICStateFields(dic_grid=dic_grid, R=10, omega_threshold=0.2, verbose_eval=True)
    dsf.tmodel_.trait_set(E=1400, c_T=0.006, nu=0.18, epsilon_0=0.0018,
                          epsilon_f=0.002, eps_max=0.01);
    dsf.eval()
    dcl = DICCrackList(dsf=dsf, corridor_left=3, corridor_right=1, x_boundary=30)
    dcl.identify_cracks()
    return dcl

## Detected cracks

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import os.path
home_dir = os.path.expanduser('~')
data_dir = os.path.join(home_dir, 'simdb', 'data', 'shear_zone')
data_file = os.path.join(data_dir, 'report_crack_list.pdf')

test_list = ['B1_TV1', 'B1_TV2', 'B7_TV1']
# test_list = ['B7_TV1']

with PdfPages(data_file) as pdf:
    for test_name in test_list:
        
        print('evaluating test {}'.format(test_name))
        cdl = get_dcl(test_name)

        plt.rc('text', usetex=False)
        fig, ax = plt.subplots(1, 1)
        
        cdl.bd.plot_sz_bd(ax)
        cdl.dsf.plot_crack_detection_field(ax, fig)
        try:
            cdl.plot_cracking_hist2(ax)
            cdl.critical_crack.plot_x_t_crc_Ka(ax, line_width=2, line_color='red', tip_color='red')
        except:
            pass
        ax.axis('equal')
        ax.axis('off');

        cdl.dsf.dic_grid.plot_bounding_box(ax)
        cdl.dsf.dic_grid.plot_box_annotate(ax)
        
        plt.title('Test {}'.format(test_name))
        ax.axis('equal')
        ax.axis('off');
        pdf.savefig(fig)
        plt.close()
        
    # We can also set the file's metadata via the PdfPages object:
    d = pdf.infodict()
    d['Title'] = 'Multipage PDF Example'
    d['Author'] = u'Jouni K. Sepp\xe4nen'
    d['Subject'] = 'How to create a multipage pdf file and set its metadata'
    d['Keywords'] = 'PdfPages multipage keywords author title subject'
    d['CreationDate'] = datetime.datetime(2009, 11, 13)
    d['ModDate'] = datetime.datetime.today()